<a id="toc"></a>
# HERMES beamline simulation with PyOptiX
***

Contents :
1. [Optical elements declaration](#def_opt)
1. [Definition of optical parameters](#def_param)
1. [Alignment scripts](#alignement)
1. [Simulation execution](#exec)
1. [Visualisation](#visu)

In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'GPL-3.0'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '23/JUL/2024'
__changed__ = '06/AUG/2024'

import ctypes

import numpy as np
import pandas as pd
import pyoptix
import pyoptix.classes as opx
from pyoptix.ui_objects import plot_spd_plotly
from scipy.constants import c, degree, eV, h, micro, milli, nano, pi, pico
from typing import List
from barc4xoc.aux_pyoptix import save_beam_data_to_csv, read_pyoptix_beam_from_csv

hc = h*c/eV

pyoptix.set_aperture_active(False)
pyoptix.output_notebook()


<a id="def_opt"></a>
## Optical elements declaration
[Back to the top](#toc)

In [ ]:
# beamline class
Hermes = opx.Beamline(name="Hermes - current")

# ------------------
# source
# ------------------
ondulator_LE = opx.UndulatorSource(name="ond_LE")         # low energy 
ondulator_HE = opx.UndulatorSource(name="ond_HE")         # high energy

# ------------------
# optical elements
# ------------------
pupil = opx.PlaneFilm(name="pupil")

# M1
m1a = opx.PlaneMirror(name="M1A")
m1b = opx.ToroidalMirror(name="M1B")
m1c = opx.ToroidalMirror(name="M1C")

# monochromator
mono_entrance_slit = opx.PlaneFilm(name="mono_in")
grating_450 = opx.PlaneHoloGrating(name="grating_450")
grating_600 = opx.PlaneHoloGrating(name="grating_600")
m2 = opx.PlaneMirror(name="M2")
m3 = opx.ToroidalMirror(name="M3")
mono_exit_slit = opx.PlaneFilm(name="mono_out")

# branch STXM
m4 = opx.ToroidalMirror(name="M4")
vs_m4 = opx.PlaneFilm(name="virtual_source_M4")

# branch PEEM
m5 = opx.CylindricalMirror(name="M5")
vs_m5 = opx.PlaneFilm(name="virtual_source_M5")
m6 = opx.ConicCylindricalMirror(name="M6")
m7 = opx.ConicCylindricalMirror(name="M7")

# ------------------
# endstations
# ------------------
peem = opx.PlaneFilm(name="PEEM")
stxm = opx.PlaneFilm(name="STXM")

In [ ]:
branches = [[mono_exit_slit, m5, vs_m5, m6, m7, peem],
            [mono_exit_slit, m4, vs_m4, stxm],
            [mono_exit_slit]]

for undulator, m1bis, grating in [(ondulator_LE, m1b, grating_450), (ondulator_HE, m1c, grating_600)]:
    for branch in branches:
        chain_name = f"{undulator.name.split('_')[1]}_G{grating.name.split('_')[1]}_{branch[-1].name}"
        Hermes.chains[chain_name] = [
            undulator, pupil, m1a, m1bis, mono_entrance_slit, grating, m2, m3
        ] + branch

for chain_name in Hermes.chains:
    print(chain_name,":\n\t",Hermes.chains[chain_name])

<a id="def_param"></a>
## Definition of optical parameters

In this section we define the static parameters of the optical elements. Characteristics that change with 
either configuration or energy are defined in the [Alignment scripts](#alignement) section.

[Back to the top](#toc)

### Low energy undulator (**HU64**)

In [ ]:
e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=4.7890, eta_x= 0.1804, etap_x= 0.0007, alpha_x=-0.3858,
                  beta_y=3.7497, eta_y=-0.0044, etap_y=-0.0025, alpha_y=-0.7746)
e_beam.print_rms()

hu64 = opx.MagneticStructure(period_length=64e-3, number_of_periods=28)

ondulator_LE.electron_beam = e_beam
ondulator_LE.magnetic_structure = hu64

ondulator_LE.write_syned_config(r".\resources\soleil_hu64","Soleil - HU64")

### High energy undulator (**HU42**)

In [ ]:
e_beam = opx.ElectronBeam()
e_beam.from_twiss(energy=2.75, energy_spread=0.1025/100, current=0.500,
                  emittance=3.94*nano, coupling=1/100,
                  beta_x=4.1825, eta_x=0.1792, etap_x= 0.0007, alpha_x=0.0578,
                  beta_y=2.3439, eta_y=0.0001, etap_y=-0.0025, alpha_y=0.0143)
e_beam.print_rms()

hu42 = opx.MagneticStructure(period_length=42e-3, number_of_periods=42)

ondulator_HE.electron_beam = e_beam
ondulator_HE.magnetic_structure = hu42

ondulator_HE.write_syned_config(r".\resources\soleil_hu42","Soleil - HU42")

### Entrance pupil
[Back to the top](#toc)

In [ ]:
pupil.distance_from_previous = 20
pupil.recording_mode = opx.RecordingMode.recording_output
pupil.next = m1a

### M1

[Back to the top](#toc)

In [ ]:
m1_grazing_angle = [2.5*degree, 1.2*degree]

m1_distance = opx.M1_triad_distances(incident_angle=m1_grazing_angle,
                                    transverse_distance=41e-3, 
                                    verbose=1)

#### - M1A

In [ ]:
m1a.distance_from_previous = 0
m1a.phi = -90*degree # rad
m1a.recording_mode = opx.RecordingMode.recording_output

#### - M1B

In [ ]:
m1b.distance_from_previous = m1_distance[0][0]
m1b.theta = m1_grazing_angle[0]
m1b.phi = 180*degree   
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.recording_mode = opx.RecordingMode.recording_output
m1b.next = mono_entrance_slit

#### - M1C

In [ ]:
m1c.distance_from_previous = m1_distance[1][0]
m1c.theta = m1_grazing_angle[1] 
m1c.phi = 180*degree   
m1c.minor_curvature = 1/0.8870043 # m-1
m1c.major_curvature = 1/229.5203 # m-1
m1c.recording_mode = opx.RecordingMode.recording_output
m1c.next = mono_entrance_slit

### Monochromator

[Back to the top](#toc)

#### Mono entrance slit

In [ ]:
mono_entrance_slit.distance_from_previous = 2.7
mono_entrance_slit.phi = -90*degree
mono_entrance_slit.recording_mode = opx.RecordingMode.recording_output

#### Grating 450 l/mm

In [ ]:
grating_450.distance_from_previous = 0.6
grating_450.line_density = 450/milli
grating_450.inverse_distance1=-0.1761099
grating_450.inverse_distance2=-0.1111111
grating_450.elevation_angle1=-min(np.arccos(0.7986355-0.157995),np.arccos(0.7986355))
grating_450.recording_wavelength=351.1*nano
grating_450.get_vls_law(200e-3 ,3)
grating_450.order_align = 1
grating_450.order_use = 1
grating_450.recording_mode = opx.RecordingMode.recording_input
grating_450.next = m2

#### Grating 600 l/mm

In [ ]:
grating_600.distance_from_previous = 0.6
grating_600.line_density = 600/milli
grating_600.inverse_distance1=-0.2002547
grating_600.inverse_distance2=-0.1111111
grating_600.elevation_angle1=-min(np.arccos(0.7986355), np.arccos(0.7986355-0.21066))
grating_600.recording_wavelength=351.1*nano
grating_600.get_vls_law(200e-3 ,3)
grating_600.order_align = 1
grating_600.order_use = 1
grating_600.recording_mode = opx.RecordingMode.recording_output
grating_600.next = m2

#### M2

In [ ]:
m2.phi = 180*degree 
m2.recording_mode = opx.RecordingMode.recording_output
m2.next = m3

#### M3

In [ ]:
m3.theta = 1.2*degree
m3.phi = -90*degree   
m3.minor_curvature = 1/0.1462124
m3.major_curvature = 1/83
m3.recording_mode = opx.RecordingMode.recording_output
m3.next = mono_exit_slit

#### Mono exit sit

In [ ]:
mono_exit_slit.distance_from_previous = 3.5
mono_exit_slit.phi = -90*degree
mono_exit_slit.recording_mode = opx.RecordingMode.recording_output

### STXM branch

[Back to the top](#toc)

#### M4

In [ ]:
m4.distance_from_previous = 2
m4.theta = 1.35*degree
m4.phi = -90*degree
m4.minor_curvature = 1/0.0396
m4.major_curvature = 1/79
m4.recording_mode = opx.RecordingMode.recording_output
m4.next = vs_m4

#### virtual source (M4)

In [ ]:
vs_m4.distance_from_previous = 1.45
vs_m4.phi = 90*degree
vs_m4.recording_mode = opx.RecordingMode.recording_output
vs_m4.next = stxm

#### entrance of the microscope (just before the zone-plate)

In [ ]:
stxm.distance_from_previous = 3.5
stxm.recording_mode = opx.RecordingMode.recording_output

### PEEM branch

[Back to the top](#toc)

#### M5

In [ ]:
m5.distance_from_previous = 1
m5.theta = 1.75*degree
m5.phi = 90*degree
m5.curvature = 1/35
m5.axis_angle = 0 
m5.recording_mode = opx.RecordingMode.recording_output
m5.next = vs_m5

#### virtual source (M5)

In [ ]:
vs_m5.distance_from_previous = 0.6187585
vs_m5.phi = -90*degree
vs_m5.recording_mode = opx.RecordingMode.recording_output
vs_m5.next = m6

#### M6 (KB - HFM)

In [ ]:
m6.distance_from_previous = 0.6+7.781242
m6.theta = 1.75*degree
m6.phi = -90*degree
m6.inverse_p = -1/(m6.distance_from_previous)
m6.inverse_q = 1/(0.5 + 1.80)
m6.theta0 = m6.theta
m6.recording_mode = opx.RecordingMode.recording_output
m6.next = m7

#### M7 (KB - VFM)

In [ ]:
m7.distance_from_previous = 0.5
m7.theta = 1.75*degree
m7.phi = 90*degree
m7.inverse_p = -1/(m7.distance_from_previous+m6.distance_from_previous+vs_m5.distance_from_previous+m5.distance_from_previous)
m7.inverse_q = 1/(1.80)
m7.theta0 = m7.theta
m7.recording_mode = opx.RecordingMode.recording_output
m7.next = peem

#### PEEM

In [ ]:
peem.distance_from_previous = 1.8
peem.recording_mode = opx.RecordingMode.recording_output

<a id="alignement"></a>
## Alignment scripts

In this section we define the all the parameters that change with either configuration or energy. These scripts are taylored
for each 

[Back to the top](#toc)

### Undulators

In [ ]:
def align_undulator(active_chain, wavelength, **kwargs):
    """
    Sets photon beam size and divergence as a function of wavelength in [m]
    """

    verbose = kwargs.get("verbose", False)
    dist_slit = kwargs.get("dist_slit", 20)
    
    if verbose:
        print("\n>>>> Aligning undulator")

    active_chain[0].set_undulator(wavelength, **kwargs)
    active_chain[1].distance_from_previous = dist_slit
    if "LE" in active_chain[0].name:
        active_chain[1].distance_from_previous -= (200.9266-199.0776)


### M1 triad

In [ ]:
def align_m1(active_chain, m1_distances, **kwargs):
    """
    Sets the M1A angle and distance from mono entrance slit to M1bis
    """
    
    dist_slit = kwargs.get("dist_slit", 2.7)
    verbose = kwargs.get("verbose", False)

    if verbose:
        print("\n>>>> Aligning M1")
   
    active_chain[2].theta = active_chain[3].theta
    active_chain[2].next  = active_chain[3]

    active_chain[4].distance_from_previous = dist_slit
    if "1B" in active_chain[3].name:
        active_chain[4].distance_from_previous += (m1_distances[1][1]-m1_distances[0][1])

    if verbose:
        name = active_chain[3].name
        angle = active_chain[3].theta/degree
        dist = active_chain[3].distance_from_previous
        print(f"M1A-{name} alignment: grazing angle {angle:.2f} degrees and {dist:.3f} m between them")

### Monochromator

In [ ]:
def align_mono(active_chain, wavelength, alignment_condition, alignment_condition_value,
               GM2_trans_dist, GM3_proj_dist, **kwargs):
    """
    Sets the grating and M2 angles and relative distances between G/M2/M3
    """

    verbose = kwargs.get("verbose", False)

    grating = None
    for oe in active_chain:
        if "reseau" in oe.name.lower() or "grating" in oe.name.lower():
            grating = oe

    if grating is None:
        raise ValueError("No grating appears in this beamline configuration")
    
    gdict = opx.align_grating(grating, verbose=0, 
                             apply_alignment=True, 
                             return_parameters=True,
                             condition=alignment_condition, 
                             condition_value=alignment_condition_value, 
                             lambda_align=wavelength, 
                             order=grating.order_align,
                             line_density=grating.line_density)
    
    gm2 = GM2_trans_dist/np.sin(gdict["deviation"])
    m2m3 = GM3_proj_dist-(GM2_trans_dist/np.tan(gdict["deviation"]))

    active_chain[6].theta = gdict["deviation"]/2
    active_chain[6].distance_from_previous = gm2

    active_chain[7].distance_from_previous = m2m3

    if verbose:
        print(f"\n>>>> {grating.name} grating alignment for a "+
              f"{alignment_condition} {alignment_condition_value} - wavelength " +
              f"{wavelength/nano:.3f} nm (E={hc/wavelength:.3f} eV)")
        print(f"> alpha {gdict['alpha_deg']:.6f} deg")
        print(f"> beta {gdict['beta_deg']:.6f} deg")
        print(f"> G-M2 distance {gm2:.6f} m")
        print(f"> theta_m2 {(gdict['alpha_deg']+gdict['beta_deg'])/2:.6f} deg")
        print(f"> M2-M3 distance {m2m3:.6f} m")

### Beamline alignment procedure

In [ ]:
def align_call(alignment_wvl, emission_wvl, alignment_condition, alignment_condition_value, 
               m1_distances, GM2_trans_dist, GM3_proj_dist, **kwargs):

    align_undulator(Hermes.active_chain, emission_wvl, **kwargs)
    align_m1(Hermes.active_chain, m1_distances, **kwargs)
    align_mono(Hermes.active_chain, alignment_wvl, alignment_condition, 
               alignment_condition_value, GM2_trans_dist, GM3_proj_dist, **kwargs)
Hermes.align_steps = align_call

def set_bl(energy_alignment, alignment_condition, alignment_condition_value, 
               m1_distances=m1_distance, GM2_trans_dist=15e-3, GM3_proj_dist=0.55, **kwargs):

    energy_radiate =  kwargs.get("energy_radiate", energy_alignment)
    dE =  kwargs.get("dE", 0)
    rays = kwargs.get("rays", 500)

    alignment_wvl = hc/energy_alignment
    emission_wvl = hc/energy_radiate

    Hermes.align(alignment_wvl, emission_wvl, 
                 alignment_condition=alignment_condition, 
                 alignment_condition_value=alignment_condition_value, 
                 m1_distances=m1_distances, GM2_trans_dist=GM2_trans_dist,
                 GM3_proj_dist=GM3_proj_dist, **kwargs)

    Hermes.clear_impacts(clear_source=True)
    Hermes.active_chain[0].nrays = rays
    if dE==0:
        Hermes.generate(emission_wvl)
    else:
        for E in np.arange(energy_radiate*(1-dE), energy_radiate*(1+dE)):
            Hermes.generate(hc/E)
    Hermes.radiate()

<a id="exec"></a>
## Simulation execution
[Back to the top](#toc)

### Available beamline configurations

In [ ]:
print("Low energy configurations:\n")
for chain_name in Hermes.chains:    
    if "450" in chain_name:
        print(chain_name,":\n\t",Hermes.chains[chain_name])

In [ ]:
print("High energy configurations:\n")
for chain_name in Hermes.chains:    
    if "600" in chain_name:
        print(chain_name,":\n\t",Hermes.chains[chain_name])

The beamline can be conviniently simulated by selecting an **active chain** and an energy:

```python
Hermes.active_chain = "LE_G450_STXM"
bl_align(500, alignment_condition="cff", alignment_condition_value=0.2, verbose=True)
```

To check the orientation of the optical elements, draw the beamline:

```python
Hermes.show_active_chain_orientation()
Hermes.draw_active_chain()
spots = Hermes.draw_to_scale()
```

<a id="visu"></a>
## Visualisations 
[Back to the top](#toc)

In [ ]:
E=719.9
number_of_rays = 5000

### HU64 with 450 l/mm grating - E=719.9 (Fe $L_2$)
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "LE_G450_mono_out"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

#### Mono entrance slit
[Back to the top](#toc)

In [ ]:
oe = mono_entrance_slit
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_mono_entrance_slit.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False,) 

#### Mono exit slit - $E_0$
[Back to the top](#toc)

In [ ]:
oe = mono_exit_slit
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_mono_exit_slit.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False,) 

#### G450 l/mm - Resolving power - $\Delta E/E=20'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/20000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP20000_p.csv")

set_bl(E, energy_radiate=E*(1-2/20000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP20000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G450 l/mm - Resolving power - $\Delta E/E=15'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/15000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP15000_p.csv")

set_bl(E, energy_radiate=E*(1-2/15000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP15000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G450 l/mm - Resolving power - $\Delta E/E=10'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/10000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP10000_p.csv")

set_bl(E, energy_radiate=E*(1-2/10000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP10000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G450 l/mm - Resolving power - $\Delta E/E=7'500$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/7500), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP7500_p.csv")

set_bl(E, energy_radiate=E*(1-2/7500), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP7500_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G450 l/mm - Resolving power - $\Delta E/E=5'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/5000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP5000_p.csv")

set_bl(E, energy_radiate=E*(1-2/5000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_RP5000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### STXM virtual source
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "LE_G450_STXM"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

In [ ]:
oe = vs_m4
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_stxm.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### PEEM fucused source
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "LE_G450_PEEM"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

In [ ]:
oe = peem
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G450_peem.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

### HU42 with 600 l/mm grating - E=719.9 (Fe $L_2$)
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "HE_G600_mono_out"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

#### Mono entrance slit
[Back to the top](#toc)

In [ ]:
oe = mono_entrance_slit
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_mono_entrance_slit.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, ) 

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False,) 

#### Mono exit slit - $E_0$
[Back to the top](#toc)

In [ ]:
oe = mono_exit_slit
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_mono_exit_slit.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False,) 

#### G600 l/mm - Resolving power - $\Delta E/E=20'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/20000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP20000_p.csv")

set_bl(E, energy_radiate=E*(1-2/20000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP20000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G600 l/mm - Resolving power - $\Delta E/E=15'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/15000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP15000_p.csv")

set_bl(E, energy_radiate=E*(1-2/15000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP15000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G600 l/mm - Resolving power - $\Delta E/E=10'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/10000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP10000_p.csv")

set_bl(E, energy_radiate=E*(1-2/10000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP10000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G600 l/mm - Resolving power - $\Delta E/E=7'500$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/7500), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP7500_p.csv")

set_bl(E, energy_radiate=E*(1-2/7500), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP7500_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### G600 l/mm - Resolving power - $\Delta E/E=5'000$
[Back to the top](#toc)

In [ ]:
set_bl(E, energy_radiate=E*(1+2/5000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP5000_p.csv")

set_bl(E, energy_radiate=E*(1-2/5000), alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_RP5000_n.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="Y", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### STXM virtual source
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "HE_G600_STXM"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

In [ ]:
oe = vs_m4
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_stxm.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

#### PEEM fucused source
[Back to the top](#toc)

In [ ]:
Hermes.active_chain = "HE_G600_PEEM"
set_bl(E, alignment_condition="cff", alignment_condition_value=0.2, verbose=True, rays=number_of_rays, center_undulator=0)

In [ ]:
oe = peem
f = opx.get_optimal_focalization_distance(oe, verbose=True)
beam = save_beam_data_to_csv(oe.get_impacts(), "./results/pyoptix_G600_peem.csv")

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="X", y_key="dX", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )

In [ ]:
opx.plot_spd_plotly(oe.get_diagram(distance_from_oe=0), oe_name=oe.name, x_key="Y", y_key="dY", 
                show_map=False, light_plot=False, orthonorm=False, save_in_file="", return_fwhm=False, )